In [5]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class TrainingConfig:
    model_name: str = "ResNet50"
    learning_rate: float = 1e-4
    batch_size: int = 32
    epochs: int = 10
    use_mixed_precision: bool = True
    hidden_layers: List[int] = field(default_factory = lambda: [512, 256])

    def __post_int__(self):
        if self.learning_rate <= 0:
            raise ValueError("Learning rate must be positive!!!")

config = TrainingConfig(learning_rate = 0.01)
print(config)

TrainingConfig(model_name='ResNet50', learning_rate=0.0001, batch_size=0.01, epochs=10, use_mixed_precision=True, hidden_layers=[512, 256])


In [15]:
# time decorator
from functools import wraps
import time

def time_deco(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"start time {time.time()}")
        tmp = func(*args, **kwargs)
        print(f"end time {time.time()}")
        return tmp
    return wrapper

@time_deco
def my_func(length:int):
    print("start function")
    for i in range(length):
        time.sleep(0.00001)
    print("end function")

my_func(100)

start time 1758113114.854131
start function
end function
end time 1758113114.8562539


In [18]:
# logger decorator
from functools import wraps
import time

def logger_deco(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        print(f"timestamp: {time.time()}")
        return None
    return wrapper

@logger_deco
def my_func(length:int):
    time.sleep(0.01)

for i in range(10):
    my_func(10)

timestamp: 1758113409.533864
timestamp: 1758113409.545376
timestamp: 1758113409.5556
timestamp: 1758113409.5673869
timestamp: 1758113409.5791008
timestamp: 1758113409.5916579
timestamp: 1758113409.604203
timestamp: 1758113409.616771
timestamp: 1758113409.62935
timestamp: 1758113409.641221


In [64]:
# seed_everyrthing decorator
# 숙제 : decorator 하나만 쓰기
from functools import wraps
import random

def rand_dec(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        value = random.random()
        print(f"random value : {value}")
        return value
    return wrapper

@rand_dec
def my_func(seed:int):
    random.seed(seed)
    print(f"seed {seed} set!!")

my_func(40)

seed 40 set!!
random value : 0.4586067093870614


0.4586067093870614

In [68]:
# decorate 세 개를 전부 써서 함수 하나 만들기

# start time 찍어줌
# seed 를 random 하게 set 해서 10회 rand 찍어줌
# 각 seed time 마다 time stamp 찍어줌
# end time 찍어줌

@time_deco
def my_func(randcount:int):
    @logger_deco
    @rand_dec
    def set_seed(randcount:int):
        seed = random.randint(0,100)
        random.seed(seed)
        print(f"seed {seed} set!!")

    for i in range(randcount):
        set_seed(randcount)

my_func(10)

start time 1758266235.609023
seed 97 set!!
random value : 0.19453724793318106
timestamp: 1758266235.6091
seed 47 set!!
random value : 0.35184625582788265
timestamp: 1758266235.609122
seed 55 set!!
random value : 0.09033985426934954
timestamp: 1758266235.609143
seed 25 set!!
random value : 0.376962302390386
timestamp: 1758266235.609164
seed 1 set!!
random value : 0.13436424411240122
timestamp: 1758266235.609185
seed 97 set!!
random value : 0.19453724793318106
timestamp: 1758266235.609205
seed 47 set!!
random value : 0.35184625582788265
timestamp: 1758266235.609226
seed 55 set!!
random value : 0.09033985426934954
timestamp: 1758266235.609248
seed 25 set!!
random value : 0.376962302390386
timestamp: 1758266235.6092749
seed 1 set!!
random value : 0.13436424411240122
timestamp: 1758266235.6092958
end time 1758266235.609301


In [85]:
# 과제
# 요구사항 : 타입 힌트 쓰기 

# @timer : 실행 시간 측정 -> 1 epoch 당 걸리는 시간 측정하는 함수
# @logger : 호출 인자와 반환값 기록 -> 디버깅 목적. 타입 힌트에서 어떤 값을 보유하고 있는지, 바뀐 값이 뭔지 기록하는 함수
# @seed_everything : 랜덤 시드 고정 -> train set 과 test set 의 분리를 동일하게 할 수 있는 재현성 목적의 함수

from functools import wraps
import time
import random
from dataclasses import dataclass, field
import typing
import pandas as pd

# deeplearning model config
@dataclass
class TrainingConfig:
    model_name : str = "ResNet50"
    learning_rate : float = 1e-4
    epoch : int = 10
    batch_size : int = 32
    use_mixed_precision : bool = False
    hidden_layers : List[int] = field(default_factory = lambda: [512, 256])
    seed : int = 42
    train_device : float = 0.8 # train set : test set = 8 : 2

    def __post_int__(self):
        if self.learning_rate <= 0 :
            raise ValueError("[ Error ] Learning rate must be positive")
        if self.epoch <= 0 :
            raise ValueError("[ Error ] Epoch must be positive")
        
def timer(func): # epoch 의 시간을 측정하는 decorator
    @wraps(func)
    def warpper(*args, **kwargs):
        start_time = time.time() # 시작 시간 
        func(*args, **kwargs) # 함수 실행
        end_time = time.time() # 종료 시간 
        t = (end_time - start_time).__round__(5)
        print(f"epoch time : {t}")
    return warpper

def logger(func): # config 를 csv 로 저장하는 decorator 
    # 나중에 업데이트하는 decorator로 변경해야 함 (새로 만들거나 지난 데이터 없애는 게 아니라...)
    @wraps(func)
    def wrapper(*args, **kwargs):
        # config 를 받아서 csv 저장 후 함수 실행
        config = func(*args, **kwargs)
        # print(f"config : {config}")
        df = pd.DataFrame([config.__dict__])
        df.to_csv("./config.csv", index=False)
    return wrapper
    
def set_data(func): # 데이터의 index를 불러오는 함수로 하고 싶은데...
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) 
    return wrapper

@set_data
def seed_everythig(seed : int): # seed 고정하는 decorator
    random.seed(seed)
    print(f"seed {seed} set!!")


@logger
def set_config(): # config 를 세팅하는 함수
    # 하나하나 하기 귀찮아서 일단 이렇게 임시로 씀
    # TODO: 이후에 하나씩 입력받을 수 있게 바꿔야 할 것 같음
    config = TrainingConfig(learning_rate = 0.01)
    return config

@timer
def data_read(iter : int): # 데이터를 읽는 함수. 하나의 epoch 를 표현하려고 함.
    for i in range(100):
        if i % 10 == 0:
            print(f"iter {i} / 100")

@set_data
def train(epoch : int): # train 함수. 
    for i in range(epoch):
        print(f"epoch {i} start ...")
        data_read(epoch)
        print(f"epoch {i} end ...")



idx = list(range(100)) # 임시 데이터 생성
set_config()
train(10)

epoch 0 start ...
iter 0 / 100
iter 10 / 100
iter 20 / 100
iter 30 / 100
iter 40 / 100
iter 50 / 100
iter 60 / 100
iter 70 / 100
iter 80 / 100
iter 90 / 100
epoch time : 5e-05
epoch 0 end ...
epoch 1 start ...
iter 0 / 100
iter 10 / 100
iter 20 / 100
iter 30 / 100
iter 40 / 100
iter 50 / 100
iter 60 / 100
iter 70 / 100
iter 80 / 100
iter 90 / 100
epoch time : 4e-05
epoch 1 end ...
epoch 2 start ...
iter 0 / 100
iter 10 / 100
iter 20 / 100
iter 30 / 100
iter 40 / 100
iter 50 / 100
iter 60 / 100
iter 70 / 100
iter 80 / 100
iter 90 / 100
epoch time : 4e-05
epoch 2 end ...
epoch 3 start ...
iter 0 / 100
iter 10 / 100
iter 20 / 100
iter 30 / 100
iter 40 / 100
iter 50 / 100
iter 60 / 100
iter 70 / 100
iter 80 / 100
iter 90 / 100
epoch time : 4e-05
epoch 3 end ...
epoch 4 start ...
iter 0 / 100
iter 10 / 100
iter 20 / 100
iter 30 / 100
iter 40 / 100
iter 50 / 100
iter 60 / 100
iter 70 / 100
iter 80 / 100
iter 90 / 100
epoch time : 4e-05
epoch 4 end ...
epoch 5 start ...
iter 0 / 100
iter 10 /